In [1]:
import pandas as pd

In [30]:
df_1 = pd.DataFrame({"Class": ["A", "A", "A", "A", "B", "B", "B", "B"],
                     "Status": ["Open", "Closed", "Open", "Closed",
                                "Open", "Closed", "Open", "Closed"]})

df_2 = pd.DataFrame({"Class": ["A", "A", "A", "A"],
                     "Status": ["Closed", "Closed", "Closed", "Closed"]})

_indexes = df_1.index[df_1["Class"] == "A"].values
df_1.loc[_indexes, "Status"] = df_2["Status"]
df_1

,Class,Status
0,A,Closed
1,A,Closed
2,A,Closed
3,A,Closed
4,B,Open
5,B,Closed
6,B,Open
7,B,Closed


## Access Ollama Server by Openai() function

#### Reference
- Structured Outputs with Ollama: https://github.com/jxnl/instructor/blob/main/docs/examples/ollama.md

In [ ]:
# !uv pip install pydantic instructor openai yfinance

In [37]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import yfinance as yf
import instructor

In [15]:
company = "Google"

class StockInfo(BaseModel):
    company: str = Field(..., description="Name of the company")
    ticker: str = Field(..., description="Ticker symbol of the company")

In [19]:
# enables `response_model` in create call
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

resp = client.chat.completions.create(
    model="llama3",
    messages=[
        {
            "role": "user",
            "content": f"Return the company name and the ticker symbol of the {company}."
        }
    ],
    response_model=StockInfo,
    max_retries=5
)

In [39]:
stock = yf.Ticker(resp.ticker)
hist = stock.history(period="1d")
stock_price = hist['Close'].iloc[-1]
print(f"The stock price of the {resp.company} is {stock_price}. USD")

The stock price of the Google LLC is 177.85000610351562. USD
